In [1]:
import pandas as pd
import numpy as np
import os
import geoio

In [2]:
BASE_DIR = '..'
NIGHTLIGHTS_DIRS = [os.path.join(BASE_DIR, 'data/nightlights/viirs_2015_00N060W.tif'),
                    os.path.join(BASE_DIR, 'data/nightlights/viirs_2015_75N060W.tif')]

COUNTRIES_DIR = os.path.join(BASE_DIR, 'data', 'countries')

In [3]:
import sys
sys.path.append(BASE_DIR)
from utils import create_space

In [4]:
def process_malawi():
    lsms_dir = os.path.join(COUNTRIES_DIR, 'malawi_2016', 'LSMS')
    consumption_file = 'consumption_aggregate/IHS4 Consumption Aggregate.csv'
    consumption_ph_col = 'rexpagg'  # per household
    hhsize_col = 'hhsize'  # people in household

    geovariables_file = 'household_geovariables/householdgeovariablesihs4.csv'
    lat_col = 'lat_modified'
    lon_col = 'lon_modified'
    ppp = 215.182 #purchasing power parity

    for file in [consumption_file, geovariables_file]:
        assert os.path.isfile(os.path.join(lsms_dir, file)), print(f'Could not find {file}')

    df = pd.read_csv(os.path.join(lsms_dir, consumption_file))
    df['cons_ph'] = df[consumption_ph_col]
    df['pph'] = df[hhsize_col]
    df['cons_ph'] = df['cons_ph'] / ppp / 365
    df = df[['case_id', 'cons_ph', 'pph']]

    df_geo = pd.read_csv(os.path.join(lsms_dir, geovariables_file))
    df_cords = df_geo[['case_id', 'HHID', lat_col, lon_col]].copy()  # Make a copy of the subset
    df_cords.rename(columns={lat_col: 'cluster_lat', lon_col: 'cluster_lon'}, inplace=True)
    df_combined = pd.merge(df, df_cords, on='case_id')
    df_combined.drop(['case_id', 'HHID'], axis=1, inplace=True)
    df_combined.dropna(inplace=True)  # Can't use NA values

    df_clusters = df_combined.groupby(['cluster_lat', 'cluster_lon']).sum().reset_index()
    df_clusters['cons_pc'] = df_clusters['cons_ph'] / df_clusters['pph']  # Divides total cluster income by people
    df_clusters['country'] = 'mw'
    return df_clusters[['country', 'cluster_lat', 'cluster_lon', 'cons_pc']]


def process_ethiopia():
    lsms_dir = os.path.join(COUNTRIES_DIR, 'ethiopia_2015', 'LSMS')
    consumption_file = 'Consumption Aggregate/cons_agg_w3.csv'
    consumption_pc_col = 'total_cons_ann'  # per capita
    hhsize_col = 'hh_size'  # people in household

    geovariables_file = 'Geovariables/ETH_HouseholdGeovars_y3.csv'
    lat_col = 'lat_dd_mod'
    lon_col = 'lon_dd_mod'
    ppp = 7.882

    for file in [consumption_file, geovariables_file]:
        assert os.path.isfile(os.path.join(lsms_dir, file)), print(f'Could not find {file}')

    df = pd.read_csv(os.path.join(lsms_dir, consumption_file))
    df['cons_ph'] = df[consumption_pc_col] * df[hhsize_col]
    df['pph'] = df[hhsize_col]
    df['cons_ph'] = df['cons_ph'] / ppp / 365
    df = df[['household_id2', 'cons_ph', 'pph']]

    df_geo = pd.read_csv(os.path.join(lsms_dir, geovariables_file))
    df_cords = df_geo[['household_id2', lat_col, lon_col]].copy()  # Make a copy of the subset
    df_cords.rename(columns={lat_col: 'cluster_lat', lon_col: 'cluster_lon'}, inplace=True)
    df_combined = pd.merge(df, df_cords, on='household_id2')
    df_combined.drop(['household_id2'], axis=1, inplace=True)
    df_combined.dropna(inplace=True)  # Can't use NA values

    df_clusters = df_combined.groupby(['cluster_lat', 'cluster_lon']).sum().reset_index()
    df_clusters['cons_pc'] = df_clusters['cons_ph'] / df_clusters['pph']  # Divides total cluster income by people
    df_clusters['country'] = 'eth'
    return df_clusters[['country', 'cluster_lat', 'cluster_lon', 'cons_pc']]

def process_nigeria():
    lsms_dir = os.path.join(COUNTRIES_DIR, 'nigeria_2015', 'LSMS')
    consumption_file = 'cons_agg_wave3_visit1.csv'
    consumption_pc_col = 'totcons'  # per capita
    hhsize_col = 'hhsize'  # people in household

    geovariables_file = 'nga_householdgeovars_y3.csv'
    lat_col = 'LAT_DD_MOD'
    lon_col = 'LON_DD_MOD'
    ppp = 95.255

    for file in [consumption_file, geovariables_file]:
        assert os.path.isfile(os.path.join(lsms_dir, file)), print(f'Could not find {file}')

    df = pd.read_csv(os.path.join(lsms_dir, consumption_file))
    df['cons_ph'] = df[consumption_pc_col] * df[hhsize_col]
    df['pph'] = df[hhsize_col]
    df['cons_ph'] = df['cons_ph'] / ppp / 365
    df = df[['hhid', 'cons_ph', 'pph']]

    df_geo = pd.read_csv(os.path.join(lsms_dir, geovariables_file))
    df_cords = df_geo[['hhid', lat_col, lon_col]].copy()  # Make a copy of the subset
    df_cords.rename(columns={lat_col: 'cluster_lat', lon_col: 'cluster_lon'}, inplace=True)
    df_combined = pd.merge(df, df_cords, on='hhid')
    df_combined.drop(['hhid'], axis=1, inplace=True)
    df_combined.dropna(inplace=True)  # Can't use NA values

    df_clusters = df_combined.groupby(['cluster_lat', 'cluster_lon']).sum().reset_index()
    df_clusters['cons_pc'] = df_clusters['cons_ph'] / df_clusters['pph']  # Divides total cluster income by people
    df_clusters['country'] = 'ng'
    return df_clusters[['country', 'cluster_lat', 'cluster_lon', 'cons_pc']]

In [5]:
df_mw = process_malawi()

In [6]:
df_eth = process_ethiopia()

In [7]:
df_ng = process_nigeria()

In [8]:
df_mw.shape, df_eth.shape, df_ng.shape

((780, 4), (523, 4), (664, 4))

In [9]:
tifs = [geoio.GeoImage(ndir) for ndir in NIGHTLIGHTS_DIRS]

In [10]:
# loading both of these into memory requires A LOT of free memory (at least 4 gigs)
# using a swapfile of size 2 GB still did not fix my issues
# instead, I knew ahead of time the 0th tif is for Malawi, and the 1st tif is for Ethiopia and Nigeria
# I'll use this to only load one tif at a time

# thankfully, the countries did not span across two tifs
tif_array = np.squeeze(tifs[0].get_data())

In [11]:
def add_nightlights(df, tif, tif_array):
    ''' 
    This takes a dataframe with columns cluster_lat, cluster_lon and finds the average 
    nightlights using a 10kmx10km box around the point
    '''
    cluster_nightlights = []
    for i,r in df.iterrows():
        min_lat, min_lon, max_lat, max_lon = create_space(r.cluster_lat, r.cluster_lon)
        
        xminPixel, ymaxPixel = tif.proj_to_raster(min_lon, min_lat)
        xmaxPixel, yminPixel = tif.proj_to_raster(max_lon, max_lat)
        assert xminPixel < xmaxPixel, print(r.cluster_lat, r.cluster_lon)
        assert yminPixel < ymaxPixel, print(r.cluster_lat, r.cluster_lon)
        if xminPixel < 0 or xmaxPixel >= tif_array.shape[1]:
            print(f"no match for {r.cluster_lat}, {r.cluster_lon}")
            raise ValueError()
        elif yminPixel < 0 or ymaxPixel >= tif_array.shape[0]:
            print(f"no match for {r.cluster_lat}, {r.cluster_lon}")
            raise ValueError()
        xminPixel, yminPixel, xmaxPixel, ymaxPixel = int(xminPixel), int(yminPixel), int(xmaxPixel), int(ymaxPixel)
        cluster_nightlights.append(tif_array[yminPixel:ymaxPixel,xminPixel:xmaxPixel].mean())
        
    df['nightlights'] = cluster_nightlights

In [12]:
add_nightlights(df_mw, tifs[0], tif_array)

In [13]:
del tif_array
import gc
gc.collect()

0

In [14]:
import psutil
psutil.virtual_memory()

svmem(total=8373600256, available=2237952000, percent=73.3, used=6135648256, free=2237952000)

In [15]:
tif_array = np.squeeze(tifs[1].get_data())

In [16]:
add_nightlights(df_eth, tifs[1], tif_array)

In [17]:
add_nightlights(df_ng, tifs[1], tif_array)

In [18]:
df_mw.head()

,country,cluster_lat,cluster_lon,cons_pc,nightlights
0,mw,-17.095150,35.217213,1.423239,0.025206
1,mw,-17.092351,35.114643,1.266204,0.000000
2,mw,-17.016698,35.079629,1.566870,0.000000
3,mw,-16.977243,35.205706,1.669245,0.008266
4,mw,-16.956385,35.168967,1.089891,0.002295


In [26]:
df_eth.head()

,country,cluster_lat,cluster_lon,cons_pc,nightlights
0,eth,3.455701,39.515994,14.854634,0.0
1,eth,3.549937,39.184234,14.312022,0.0
2,eth,3.864243,39.101366,12.470145,0.0
3,eth,3.982931,38.491368,8.336539,0.0
4,eth,4.048194,41.930928,4.762526,0.0


In [27]:
df_ng.head()

,country,cluster_lat,cluster_lon,cons_pc,nightlights
0,ng,4.315786,6.268753,4.317717,0.123354
1,ng,4.328719,6.308172,4.880711,0.013713
2,ng,4.398427,7.183962,8.767258,38.470989
3,ng,4.425192,7.166935,10.774504,40.519035
4,ng,4.619377,7.684946,5.191333,0.000000


In [28]:
df_mw['nightlights'].mean()

0.6038163

In [29]:
df_eth['nightlights'].mean()

0.6727544

In [30]:
df_ng['nightlights'].mean()

1.6584013

In [31]:
# Assuming 'country' is a non-numeric column
df_mw_numeric = df_mw.drop('country', axis=1)
correlation_matrix = df_mw_numeric.corr()
correlation_matrix

,cluster_lat,cluster_lon,cons_pc,nightlights
cluster_lat,1.000000,-0.702793,-0.026563,-0.083273
cluster_lon,-0.702793,1.000000,-0.002947,-0.033367
cons_pc,-0.026563,-0.002947,1.000000,0.384939
nightlights,-0.083273,-0.033367,0.384939,1.000000


In [32]:
df_eth_numeric = df_eth.drop('country', axis=1)
correlation_matrix = df_eth_numeric.corr()
correlation_matrix

,cluster_lat,cluster_lon,cons_pc,nightlights
cluster_lat,1.000000,0.157130,-0.052673,-0.006627
cluster_lon,0.157130,1.000000,0.211846,0.076039
cons_pc,-0.052673,0.211846,1.000000,0.343879
nightlights,-0.006627,0.076039,0.343879,1.000000


In [33]:
df_ng_numeric = df_ng.drop('country', axis=1)
correlation_matrix = df_ng_numeric.corr()
correlation_matrix

,cluster_lat,cluster_lon,cons_pc,nightlights
cluster_lat,1.000000,0.330910,-0.376356,-0.188281
cluster_lon,0.330910,1.000000,-0.246189,-0.196671
cons_pc,-0.376356,-0.246189,1.000000,0.264398
nightlights,-0.188281,-0.196671,0.264398,1.000000


In [34]:
for country in ['malawi_2016', 'ethiopia_2015', 'nigeria_2015']:
    os.makedirs(os.path.join(COUNTRIES_DIR, country, 'processed'), exist_ok=True)

In [35]:
df_mw.to_csv(os.path.join(COUNTRIES_DIR, 'malawi_2016', 'processed/clusters.csv'), index=False)

In [36]:
df_eth.to_csv(os.path.join(COUNTRIES_DIR, 'ethiopia_2015', 'processed/clusters.csv'), index=False)

In [37]:
df_ng.to_csv(os.path.join(COUNTRIES_DIR, 'nigeria_2015', 'processed/clusters.csv'), index=False)

In [38]:
total_clusters_mw = df_mw.shape[0]
total_clusters_eth = df_eth.shape[0]
total_clusters_ng = df_ng.shape[0]

print(f"Total Clusters in Malawi: {total_clusters_mw}")
print(f"Total Clusters in Ethiopia: {total_clusters_eth}")
print(f"Total Clusters in Nigeria: {total_clusters_ng}")


Total Clusters in Malawi: 780
Total Clusters in Ethiopia: 523
Total Clusters in Nigeria: 664


In [39]:
def calculate_cluster_area(lat, lon):
    min_lat, min_lon, max_lat, max_lon = create_space(lat, lon)
    area_km2 = (max_lat - min_lat) * (max_lon - min_lon)
    return area_km2
lat, lon = df_mw.iloc[0]['cluster_lat'], df_mw.iloc[0]['cluster_lon']
area_cluster_mw = calculate_cluster_area(lat, lon)
print(f"Area of the first cluster in Malawi: {area_cluster_mw} square kilometers")

Area of the first cluster in Malawi: 0.008069703496826815 square kilometers


In [40]:
import math

# Given area in square kilometers
area_km2 = 0.008069703496826815

# Convert to linear measurement (side length of a square) in kilometers
side_length_km = math.sqrt(area_km2)

print(f"The side length of a square with an area of {area_km2} square kilometers is approximately {side_length_km} kilometers.")


The side length of a square with an area of 0.008069703496826815 square kilometers is approximately 0.08983152841194908 kilometers.
